In [ ]:
# import statements
import os

import mdtraj as md
import numpy as np
import glob
from scipy.stats import gaussian_kde


%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from functools import partial
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [ ]:
trajdict = {
    '4w51-apo-ff14': 'md-trajectory.dcd'
    
}

topdict = {
    '4w51-apo-ff14': 'complex_topology.pdb'
    
}

In [ ]:
trajname = 'md-trajectory.dcd'

plot_title = trajname

trajfile = trajname

topfile = 'complex_topology.pdb'

print(trajfile)
print(topfile)



save_as_pdf = False

In [ ]:
# load the trajectory

traj = md.load( trajfile, top=topfile )

print( traj )

In [ ]:
resids, angles = md.compute_chi1( traj )

#indices of the atoms forming the Chi1
resInds = [1733, 1735, 1737, 1739]

#print(angles)

print("The shape should be ( num_residues, 4 )")
len(angles)

In [ ]:
try:
    topology = md.load( 'complex_topology.pdb' ).topology
except:
    topology = md.load_prmtop( topfile )
    
resOfInterest = np.where(resids == resInds)

ind = resOfInterest[0][0]
print(ind)

res = ind

In [ ]:
res_angle = [0]*len(angles)
data = res_angle
    
for fnum, frame in enumerate(angles):
    res_angle[fnum] = ( 180 / 3.14 ) * angles[fnum][res]

        
for ind in range(len(res_angle)):
    if res_angle[ind] < 0:
        res_angle[ind] = 360 + res_angle[ind]

# print out the angle_degrees to make sure there are no negative values
print("Angles for residues of interest: ( in the form of RESID: [angles in degrees] )")

print('    {}: {}'.format( res, len(angles[res]) )  )

In [ ]:
def to_percent(y, position, n):
    # Ignore the passed in position. This has the effect of scaling the default
    # tick locations.
    s = str(round(100 * y / n, 3))

    if matplotlib.rcParams['text.usetex']:
        return s + r'$\%$'

    return s + '%'

In [ ]:
percent_formatter = partial(to_percent,
                            n=len(data))
formatter = FuncFormatter(percent_formatter)

In [ ]:
fig, ax1 = plt.subplots()

ax1.set_title('Val 111'.format(plot_title, topology.residue( res )), fontsize=16)
ax1.set_xlabel('Dihedral ($^\circ$)', fontsize=16)

# set y axis 1 (left) for the histogram frequency
color1 = 'steelblue'
ax1.set_ylabel('Frequency (%)', color=color1, fontsize=16)  # we already handled the x-label with ax1
bin_size = 5; min_edge = 0; max_edge = 360
N = int((max_edge  -min_edge) / bin_size)
Nplus1 = N + 1
bin_list = np.linspace(min_edge, max_edge, Nplus1)

ax1.hist(data, bins=bin_list, color='skyblue')

percent_formatter = partial(to_percent,
                            n=len(data))
formatter = FuncFormatter(percent_formatter)

#formatter = ticker.FuncFormatter(to_percent)

ax1.yaxis.set_major_formatter(formatter)

ax1.tick_params(axis='y', labelcolor=color1)




# Set y axis 2 (right) for the gaussian KDE Density
ax2 = ax1.twinx()

color2 = 'dimgrey'
ax2.set_ylabel('Density', color=color2, fontsize=16)

density = gaussian_kde( data )
xs = np.linspace( 0, 360 )
density.covariance_factor = lambda : .25
density._compute_covariance()
ax2.plot(xs, density(xs), color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

fig.tight_layout()


plt.show()